In [1]:
import os

import tifffile as tf
import pandas as pd
import numpy as np

import scipy.ndimage as ndi

import line_utils
import image_utils

In [ ]:
# Establish the path the Excel file
# base_path = r"J:\Analyses\MyosinIIA"
# base_path = r"J:\Analyses\20240924 MKLP1"
base_path = "/Volumes/Ries_Ewers/01_Macros_Analysis"
# workbook = "MKLP1_along.xlsx"
# workbook = "RacGAP1_along.xlsx"
# workbook = "20241030_anillin_along.xlsx"
# workbook = "20241111_myosinIIA_along.xlsx"
workbook = "20241107_myoIIB_along.xlsx"

# workbook = "20241008_septin2_tubulin.xlsx"
workbook_path = os.path.join(base_path, workbook)
toc_sheet_name = "ToC"
# toc_sheet_name = "ToC + P-t-p"

# Where are the referenced images located?
# image_directory = r"H:\MKLP1"
# image_directory = "/Volumes/Ries_Ewers/MKLP1"
# image_directory = '/Volumes/Ries_Ewers/RacGAP1'
# image_directory = "/Volumes/Ries_Ewers/Septin2-GFP"
# image_directory = '/Volumes/Ries_Ewers/Anillin'
# image_directory = '/Volumes/Ries_Ewers/Myosin IIA'
image_directory = '/Volumes/Ries_Ewers/Myosin IIB'

stage_key = "Stage"
# stage_key = "Unnamed: 0"
head_row = 1  # From where do we start reading

# Channels per image
n_ch = 4

# channel options
septin_ch_names = ["GFP"]
mt_ch_names = ["MTs", "aTub", "a-tub", "atub", "tub"]

# wavelengths to be found in the file names
# wvls = [488,568,647]
wvls = [488,568,[646,647]]

# expected channel order
# expected_channel_order = [mt_ch_names, "MKLP1", septin_ch_names, "DAPI"]
# expected_channel_order = [mt_ch_names, "RacGAP1", septin_ch_names, "DAPI"]
# expected_channel_order = [mt_ch_names, "anillin", septin_ch_names, "DAPI"]
# expected_channel_order = [mt_ch_names, "myoIIA", septin_ch_names, "DAPI"]
# expected_channel_order = [mt_ch_names, "anillin", septin_ch_names, "DAPI"]
expected_channel_order = [mt_ch_names, ["myoIIB", "mypoIIB", "myosinIIB"], septin_ch_names, "DAPI"]

# Length of cropped pseudotime region (should be roughly the line length)
length = 500

# Order of stages
#true_order = ["RC", "CS", "RS", "SM", "BA", "A"]
true_order = ["RC", "CS", "RS", "SM", "BA"]

In [3]:
# Load the table of contents
metrics = pd.read_excel(workbook_path, sheet_name=toc_sheet_name, header=head_row)
# toc = pd.read_excel(workbook_path, sheet_name=toc_sheet_name, header=head_row)

In [4]:
# load FWHM_along, which contains other statistics
# only for septin2_MTs worksheet
# metrics = pd.read_excel(workbook_path, sheet_name="FWHM_along", header=0).dropna(axis=0)

In [5]:
# metrics = line_utils.merge_df_information(toc, 
#                                         metrics, 
#                                         id_key="Label", 
#                                         mapped_keys=["dX (µm)", "dX (pxl)", "X1", "X2", "Angle","X","Y","length"])
metrics = metrics[~metrics['Y'].isna()]
# metrics = metrics.dropna()

KeyError: 'Y'

In [ ]:
# Now let's get the original images...
import glob

# ----- max proj files --- #
# max_proj_path = "/Volumes/Ries_Ewers/Septin2-GFP/MaxIPs"
# max_proj_files = glob.glob(max_proj_path+"/*.tif")

# # ...and associate the file with each metrics entry
# metrics["filename"] = ""
# for i, ml in metrics.iterrows():
#     for fn in max_proj_files:
#         if ml["Label"] in fn:
#             metrics.loc[i, "filename"] = fn
#             break

# ---- 3D files ---- #

image_files = glob.glob(image_directory+"/*.nd")

for i, ml in metrics.iterrows():
    file_stub = os.path.splitext(ml["Label"])[0]
    for fn in image_files:
        if file_stub in fn:
            metrics.loc[i, "filename"] = fn
            break

In [ ]:
groups = metrics.groupby(stage_key)

plot_stack = None
n_groups = len(groups)
l2 = length // 2
group_img = np.zeros((n_groups, n_ch, length, length))
metrics['dX2'] = np.nan
metrics['X12'] = np.nan
metrics['X22'] = np.nan
for group, tup in enumerate(groups):
    name, entries = tup
    n_group = len(entries)
    # mean_dX = entries['dX (pxl)'].mean()
    im_proj = {}
    for i, ml in entries.iterrows():
        # Get the image associated with this row

        # ---- If max proj -----
        # im = tf.imread(ml["filename"])
        # ---- Elif mean proj ----
        # grab the target names from the file names
        wvls_dict, binned_wvls = image_utils.extract_channel_targets_from_filename(ml["filename"], wvls=wvls)

        im = image_utils.NDImage(ml["filename"])

        # ensure the wavelengths go from high to low
        channel_targets = [wvls_dict[str(wvl)] for wvl in sorted(binned_wvls)[::-1]]

        if len(channel_targets) < n_ch:
            channel_targets.append("DAPI")  # the last channel is always DAPI, if unknown

        # Resort the image so the channels always go from high to low
        sorted_ch = np.argsort([int(x.split('CSU')[1].split(' ')[0]) for x in im.channel_names])[::-1]

        channel_order = []
        for ch in expected_channel_order:
            if isinstance(ch, list):
                for opt in ch:
                    try:
                        channel_order.append(channel_targets.index(opt))
                    except ValueError:
                        pass
            else:
                channel_order.append(channel_targets.index(ch))
        assert len(channel_order) == len(expected_channel_order)

        # Since expected channel order searches channel_targets expecting them high to low,
        # resort channels sorted high to low based on the expected order
        channel_order = sorted_ch[channel_order]
        print(os.path.basename(ml["filename"]), im.channel_names, channel_targets, channel_order)


        im = im[:].mean(1).squeeze()
        im_proj[i] = im[channel_order,...]
        # --- End If ----

        # get x, y, angle for this row
        x, y, angle = ml[["X", "Y", "Angle"]]

        # If we are in a class where it makes sense...
        if (ml[stage_key] != "A") and (ml[stage_key] != "BA"):
            # ... get the septin peaks
            mt_ch = sorted_ch[[i for i, t in enumerate(channel_targets) if any([t == n for n in mt_ch_names])][0]]
            septin_ch = sorted_ch[[i for i, t in enumerate(channel_targets) if any([t == n for n in septin_ch_names])][0]]
            p0, p1, dX2 = line_utils.find_septin_peaks(im, x, y, angle, length,
                                                        mt_ch=mt_ch, 
                                                        septin_ch=septin_ch)

            metrics.loc[i,['X12','X22','dX2']] = [p0, p1, dX2]


    mean_dX2 = entries['dX2'].mean()

    for i, ml in entries.iterrows():
        # Get the image associated with this row
        # ---- If max proj -----
        # im = tf.imread(ml["filename"])
        # ---- Elif mean proj ----
        im = im_proj[i]
        # --- End If ----
        im = im/im.sum(-1).sum(-1)[:,None, None]

        # get x, y, angle for this row
        x, y, angle = ml[["X", "Y", "Angle"]]

        # Rotate the image  # CYX
        im_rot = image_utils.pad_rot_and_trans_im(im, angle, x, y)

        # Crop the image
        xc, yc = im_rot.shape[2]//2, im_rot.shape[1]//2
        im_crop = im_rot[:,(yc-length):(yc+length),(xc-length):(xc+length)]

        # rescale the image
        # if np.isnan(ml["dX (pxl)"]):
        if np.isnan(ml["dX2"]):
            im_zoom = im_crop
        else:
            # mag = ml["dX (pxl)"]/mean_dX
            mag = ml["dX2"]/mean_dX2
            im_zoom = ndi.zoom(im_crop, (1,1,mag))

        # Crop the image again
        xc, yc = im_zoom.shape[2]//2, im_crop.shape[1]//2
        im_crop2 = im_zoom[:,(yc-l2):(yc+l2),(xc-l2):(xc+l2)]

        # Add the image with a weighting 1/length of the group 
        group_img[group,...] += (im_crop2/n_group)    

20240506_NRK49FS2GFP_UExM_GFP-488_anillin-568_atub-647_80min_06.nd ['CSU635', 'CSU561', 'CSU491', 'CSU405 QUAD'] ['atub', 'anillin', 'GFP', 'DAPI'] [0 1 2 3]
20240506_NRK49FS2GFP_UExM_GFP-488_anillin-568_atub-647_80min_05.nd ['CSU635', 'CSU561', 'CSU491', 'CSU405 QUAD'] ['atub', 'anillin', 'GFP', 'DAPI'] [0 1 2 3]
20240506_NRK49FS2GFP_UExM_GFP-488_anillin-568_atub-647_80min_04.nd ['CSU635', 'CSU561', 'CSU491', 'CSU405 QUAD'] ['atub', 'anillin', 'GFP', 'DAPI'] [0 1 2 3]
20240506_NRK49FS2GFP_UExM_GFP-488_anillin-568_atub-647_80min_03.nd ['CSU635', 'CSU561', 'CSU491', 'CSU405 QUAD'] ['atub', 'anillin', 'GFP', 'DAPI'] [0 1 2 3]
20240506_NRK49FS2GFP_UExM_GFP-488_anillin-568_atub-647_80min_02.nd ['CSU635', 'CSU561', 'CSU491', 'CSU405 QUAD'] ['atub', 'anillin', 'GFP', 'DAPI'] [0 1 2 3]
20240506_NRK49FS2GFP_UExM_GFP-488_anillin-568_atub-647_80min_01.nd ['CSU635', 'CSU561', 'CSU491', 'CSU405 QUAD'] ['atub', 'anillin', 'GFP', 'DAPI'] [0 1 2 3]
20240430_NRK49FS2GFP_UExM_GFP-488_anillin-568_atub-6

In [ ]:
# true_order = ["RC", "CS", "RS", "SM", "BA"]#, "A"]
group_order = list(groups[stage_key].unique().keys())
group_img_sorted = [group_order.index(g) for g in true_order]
print(group_img_sorted)

[3, 2, 4, 5, 1, 0]


In [ ]:
stack_fn = f'pseudotime_images_{os.path.splitext(workbook)[0]}.ome.tif'
tf.imwrite(stack_fn, group_img[group_img_sorted,...], metadata={'axes': 'TCYX'}, dtype=group_img.dtype)